In [1]:
import sys
sys.path.append('../')

In [2]:
import numpy as np
import os
import torch
from scipy.optimize import minimize
from torch.utils.data import DataLoader

from src.model import SindyModel
from src.main import get_args
from src.dataloader import ScrewdrivingDataset

In [3]:
ARGS = get_args()
data_dir = os.path.join('..', 'out', 'sindy-data')
model_dir = os.path.join('..', 'out', 'sindy-model-out', 'checkpoints')
model_checkpoints = sorted(filter(lambda f: f.endswith('.pth'), os.listdir(model_dir)))
print(f'found {len(model_checkpoints)} models')

In [4]:
model_eval_dict = {}
test_x = [batch[0].numpy() for batch in DataLoader(ScrewdrivingDataset(mode='test', **{**vars(ARGS), 'data_dir': data_dir}))]
test_x = np.concatenate(test_x, axis=0)
test_x = test_x.reshape(test_x.shape[0], test_x.shape[-1])
test_y = [batch[1].numpy() for batch in DataLoader(ScrewdrivingDataset(mode='test', **{**vars(ARGS), 'data_dir': data_dir}))]
test_y = np.concatenate(test_y, axis=0)
test_y = test_y.reshape(test_y.shape[0], test_y.shape[-1])

In [5]:
for model_name in model_checkpoints:
    with torch.no_grad():
        model = SindyModel(**vars(ARGS))
        model.load_state_dict(torch.load(os.path.join(model_dir, model_name)))
        model.eval()
        
        result = minimize(signal_error, initial_guess, args=(reference_signal, target_signal), method='Nelder-Mead')
        